In [2]:
from z3 import *

B = BoolSort()
Z = IntSort()
R = RealSort()
x,y,z = Reals('x y z')

def lemma(thm, by=[], admit=False):
    if not admit:
        #print(Implies(And(by), thm).sexpr())
        prove(Implies(And(by), thm))
    else:
        print("Admitted!!", thm)
    return thm
    
plus = Function("plus", R,R,R)
plus_def = ForAll([x, y], plus(x, y) == x + y)

plus_0 = lemma(ForAll([x], plus(x, 0) == x), by=[plus_def])
plus_comm = lemma(ForAll([x, y], plus(x, y) == plus(y, x)), by=[plus_def])
plus_assoc = lemma(ForAll([x, y, z], plus(x, plus(y, z)) == plus(plus(x, y), z)), by=[plus_def])
                   
mul = Function("mul", R,R,R)
mul_def = ForAll([x, y], mul(x, y) == x * y)

mul_zero = lemma(ForAll([x], mul(x, 0) == 0), by=[mul_def])
mul_1 = lemma(ForAll([x], mul(x, 1) == x), by=[mul_def])
mul_comm = lemma(ForAll([x, y], mul(x, y) == mul(y, x)), by=[mul_def])
mul_assoc = lemma(ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)), by=[mul_def])
mul_distrib = lemma(ForAll([x, y, z], mul(x, plus(y, z)) == plus(mul(x, y), mul(x, z))), by=[mul_def, plus_def])        

Nat = Datatype("Nat")
Nat.declare('zero')
Nat.declare('succ', ('pred', Nat))
Nat = Nat.create()
n,m,k = Consts('n m k', Nat)

real = Function("real", Nat, R)
real_def = ForAll([n], real(n) == If(Nat.is_zero(n), 0, plus(1, real(Nat.pred(n)))))


def induct(P):
    return Implies(And(P(Nat.zero), ForAll([n], Implies(P(n), P(Nat.succ(n))))),
                   #-----------------------------------------------------------
                   ForAll([n], P(n)))


proved
proved
proved
proved
proved
proved
proved
proved


In [54]:
pred = Function("pred", R, R)
pred_def = ForAll([x], pred(x) == x - 1)

ceil = Function("ceil", R, Nat)
ceil_def = ForAll([x], ceil(x) == If(x <= 0, Nat.zero, Nat.succ(ceil(x-1))))

real_zero = lemma(real(Nat.zero) == 0, by=[real_def])
ceil_real_base = lemma(ceil(real(Nat.zero)) == Nat.zero, by=[ceil_def, real_zero])
ceil_real_induct = lemma(ForAll([n], Implies(ceil(real(n)) == n, ceil(real(Nat.succ(n))) == Nat.succ(n))),
                          by=[real_def, ceil_def, plus_def])
ceil_real = lemma(ForAll([n], ceil(real(n)) == n), by=[real_def, ceil_def, plus_def, induct(lambda n: ceil(real(n)) == n)])


le = Function("le", Nat, Nat, B)
le_def = ForAll([n, m], le(n, m) == If(Nat.is_zero(n), True, If(Nat.is_zero(m), False, le(Nat.pred(n), Nat.pred(m)))))

le_refl_base = lemma(le(Nat.zero, Nat.zero), by=[le_def])
le_refl_induct = lemma(ForAll([n], Implies(le(n, n), le(Nat.succ(n), Nat.succ(n)))), by=[le_def])
le_refl = lemma(ForAll([n], le(n, n)), by=[le_refl_base, le_refl_induct, induct(lambda n: le(n, n))])

le_succ = lemma(ForAll([n,m], le(n, m) == le(Nat.succ(n), Nat.succ(m))), 
                by=[le_def, induct(lambda n: le(n, m) == le(Nat.succ(n), Nat.succ(m)))])


real_non_neg_base = lemma(real(Nat.zero) >= 0, by=[real_zero])
real_non_neg_induct = lemma(ForAll([n], Implies(real(n) >= 0, real(Nat.succ(n)) >= 0)), by=[real_def, plus_def])
real_non_neg = lemma(ForAll([n], 0 <= real(n)), by=[real_non_neg_base, real_non_neg_induct, induct(lambda n: real(n) >= 0)])


_1 = lemma(real(Nat.zero) == 0, by=[real_def])
real_le_base = lemma(ForAll([m], real(Nat.zero) <= real(m)), by=[_1, real_def, real_non_neg])
#real_le_induct = lemma(ForAll([n,m], Implies(Implies(le(n,m), real(n) <= real(m)), 
#                                             Implies(le(Nat.succ(n), m), real(Nat.succ(n)) <= real(m)))),
#                        by=[real_def, le_def, plus_def])
#real_le_base = lemma(ForAll([m], Implies(le(Nat.zero, m), real(Nat.zero) <= real(m))), by=[le_def, real_def, plus_def, real_non_neg])
#real_le_base = lemma(ForAll([n], le(Nat.zero, Nat.succ(n))), by=[le_def])
real_le = lemma(ForAll([n,m], le(n,m) == (real(n) <= real(m))),
                 by=[real_def, le_def, plus_def, induct(lambda n: Implies(le(n,m), real(n) <= real(m)))], admit=True) 


# hmm. This is probably not true, x = -1, n = 0
#ceil_galois = lemma(le(ceil(x), n) == (x <= real(n))



proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
proved
Admitted!! ForAll([n, m], le(n, m) == (real(n) <= real(m)))


In [38]:
## N = ceil(1/eps) => n > N => 1/real(n) < eps
# x <= real(ceil(x))

#real_ceil = lemma(ForAll([x], x <= real(ceil(x))), by=[real_def, ceil_def, plus_def])
# x <= real(If(x <= 0, 0, Nat.succ(real(x-1))))
real_ceil_neg = lemma(ForAll([x], Implies(x <= 0, x <= real(ceil(x)))), by=[real_def, ceil_def, plus_def])

# x <= real(Nat.succ(ceil(x-1)))) == x <= 1 + real(ceil(x-1)) ==
#real_ceil_pos = lemma(ForAll([x], Implies(x > 0, x <= real(ceil(x))), by=[real_def, ceil_def, plus_def, real_non_neg]))

# I'm pretty puzzled what principle is gonna let me do this. real induction? Some roundabout use of regular induction?
real_ceil = lemma(ForAll([x], x <= real(ceil(x))), by=[real_def, ceil_def, plus_def], admit=True)



proved
Admitted!! ForAll(x, x <= real(ceil(x)))


In [64]:
eps = Real('eps')

eps_ceil_pos = lemma(ForAll([eps], Implies(eps > 0, Not(Nat.is_zero(ceil(eps))))), by=[ceil_def])
nonzero_real_pos = lemma(ForAll([n], Implies(Not(Nat.is_zero(n)), real(n) > 0)),  by=[real_def, plus_def, real_non_neg])

epsrecip = lemma(ForAll([eps], Implies(eps > 0, 1/eps <= real(ceil(1/eps)))), by=[real_ceil])

lemma(ForAll([eps], Implies(eps > 0, 1/real(ceil(1/eps)) <= eps)), by=[epsrecip])
_1 = lemma(ForAll([eps], Implies(eps > 0, 1/(1 + real(ceil(1/eps))) < eps)), by=[real_ceil]) # so really it didn't need much help

recip_n = lemma(ForAll([eps,n], Implies(And(le(ceil(1/eps), n), eps > 0), 1/(1 + real(n)) < eps)), by=[real_ceil, real_le])
N = Const('N', Nat)

temp = Function("temp", R, Nat, Nat, B)
temp_def = ForAll([eps, N, n], temp(eps, N, n) == Implies(And(le(N, n), eps > 0), 1/(1 + real(n)) < eps))

# We can make a temporary definition to make the pattern more obvious. (It is tough on the eye, but also it's tough on z3's pattern matcher I guess)
# This is also why I suggest working in skolemized form in any case.
_1 = lemma(ForAll([eps, n], temp(eps, ceil(1/eps), n)), by=[temp_def, recip_n])
_2 = lemma(ForAll([eps], Exists([N], ForAll([n], temp(eps, N, n)))), by=[_1])
recip_n2 = lemma(ForAll([eps], Exists([N], ForAll([n], Implies(And(le(N, n), eps > 0), 1/(1 + real(n)) < eps)))), by=[_2, temp_def])


proved
proved
proved
proved
proved
proved
proved
proved
proved


Archimedean property
x = real(floor(x)) + frac(x)
Eudoxus reals?


In [ ]:

Seq = ArraySort(Nat, R)
sum_ = Function("sum", Seq, Nat, R)
s = Const('s', Seq)
sum_def = ForAll([s, n], sum_(s, n) == If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum_(s, Nat.pred(n)))))

szero = K(Nat, RealVal(0))

#sum_const_base = lemma(sum_(szero, Nat.zero) == 0, by=[sum_def, plus_def])
#sum_const_induct = lemma(ForAll([n], Implies(sum_(szero, n) == 0, 
#                                             sum_(szero, Nat.succ(n)) == 0)),
#                         by=[sum_def, plus_def])
sum_szero = lemma(ForAll([n], sum_(szero,n) == 0), by=[sum_def, plus_def, induct(lambda n: sum_(szero, n) == 0)])


# note I original wrote ForAll([x], sum_(K(Nat, x), Nat.zero) == 0) which is not thed correct syntactic hypothesis.
# it wouldn't go through.
sum_const_base = lemma(ForAll([x], sum_(K(Nat, x), Nat.zero) == mul(real(Nat.zero), x)), 
                       by=[sum_def, mul_def, real_def])
sum_const_induct = lemma(ForAll([n], Implies(ForAll([x], sum_(K(Nat, x), n) == mul(real(n), x)), 
                                             ForAll([x], sum_(K(Nat, x), Nat.succ(n)) == mul(real(Nat.succ(n)), x)))),
                         by=[sum_def, plus_def, mul_def, real_def])
sum_const = lemma(ForAll([n, x], sum_(K(Nat, x), n) == mul(real(n), x)), 
                  by=[sum_const_base, sum_const_induct,
                      induct(lambda n: ForAll([x], sum_(K(Nat, x), n) == mul(real(n),x)))])

#sum_const_induct
#induct(lambda n: ForAll([x], sum_(K(Nat, x), n) == mul(real(n),x)))
#lemma(sum_const, by=[sum_const])

id_ = Lambda([n], real(n))
# helper function. We could perhaps have4 lambdified the sympy result?
nn1 = lambda n: mul(real(n) - 1, real(n)) / 2 

sum_n_base = lemma(ForAll([n], sum_(id_, Nat.zero) == nn1(Nat.zero)), by=[sum_def, plus_def, mul_def, real_def])
sum_n_induct = lemma(ForAll([n], Implies(sum_(id_, n) == nn1(n), 
                                        sum_(id_, Nat.succ(n)) == nn1(Nat.succ(n)))),            
                     by=[sum_def, plus_def, mul_def, real_def])

sum_n = lemma(ForAll([n], sum_(id_, n) == nn1(n)), by=[sum_n_base, sum_n_induct, induct(lambda n: sum_(id_, n) == nn1(n))])

In [122]:
import sympy as sp
import sympy.abc as abc

# sum is inclusive of boundaries.
sp.Sum(abc.n, (abc.n, 0, abc.m-1)).doit()
#print(sp.Sum(1, (abc.n, 0, 1)).doit())

m**2/2 - m/2

We expect it to often be the case that sympy closed forms will be confirmable via a rote induction procedure.

In [3]:
abs = Function("abs", R, R)
abs_def = ForAll([x], abs(x) == If(x >= 0, x, -x))

abs_neg_x = lemma(ForAll([x], abs(-x) == abs(x)), by=[abs_def])
abs_pos = lemma(ForAll([x], abs(x) >= 0), by=[abs_def])

dist = Function("dist", R, R, R)
dist_def = ForAll([x, y], dist(x, y) == abs(x - y))

dist_refl = lemma(ForAll([x], dist(x, x) == 0), by=[dist_def, abs_def])
dist_post = lemma(ForAll([x, y], dist(x, y) >= 0), by=[dist_def, abs_pos])
dist_symm = lemma(ForAll([x, y], dist(x, y) == dist(y, x)), by=[dist_def, abs_def])
dist_tri = lemma(ForAll([x, y, z], dist(x, z) <= plus(dist(x, y), dist(y, z))), by=[dist_def, plus_def, abs_def])


proved
proved
proved
proved
proved
proved


In [138]:
# https://en.wikipedia.org/wiki/Cauchy_sequence
cauchy = Function("cauchy", Seq, B)
eps = Real('eps')
N = Const("N", Nat)
cauchy_def = ForAll([s], cauchy(s) == 
                    ForAll([eps], Exists([N], ForAll([n, m], Implies(And(n >= N, m >= N), abs(s[n] - s[m]) < eps)))))



SyntaxError: incomplete input (589912192.py, line 5)

In [156]:
conv_to = Function("conv_to", Seq, R, B)
eps = Real('eps')
N = Const("N", Nat)
conv_to_def = ForAll([s, x], conv_to(s, x) == ForAll([eps], Exists([N], ForAll([n], Implies(real(n) >= real(N), abs(s[n] - x) < eps)))))

converges = Function("converges", Seq, B)
converges_def = ForAll([s], converges(s) == Exists([x], conv_to(s, x)))

lim = Function("lim", Seq, R)
lim_def = ForAll([s], Implies(converges(s), conv_to(s, lim(s))))

recip_converge = lemma(conv_to(Lambda([n], 1/(1 + real(n))), 0), by=[conv_to_def, real_def], admit=True)

# constructing the appropriate N(eps) is annoying
#abs(Lambda([n], 1/(1 + real(n)))   
#round_up = Function("round_up", R, Nat)
#round_up_def = ForAll([x], round_up(x) == If(x <= 0, 1, Nat.pred(round_up(x))), Nat.zero))





failed to prove


Z3Exception: model is not available

In [ ]:
mono = Function("mono", Seq, B)
mono_def = ForAll([s], mono(s) == ForAll([n], s[n] <= s[Nat.succ(n)]))

bounded = Function("bounded", Seq, B)
bounded_def = ForAll([s], bounded(s) == Exists([y], ForAll([n], s[n] <= y)))

RNat = Function("Nat", R, Bool) # Nat as a predictae on R
RNat(x) == (real(ceil(x)) == x) # In terms of previous stuff.

RNat is the least set that contains 0 and RNat(x) => RNat(x+1)

Direct rnat induction

----------------------------------
ForAll([x], Implies(RNat(x), P(x))


Real induction. If it is uniform real induction, we can push out a ball for an epsilon that does not depend on the size of the ball.
P(B) /\ exists eps, forall B, P(B) -> P(B+eps)
--------------------------------------
      forall x, P(x) 

This is sort of like the reverse of a monotonic convergence. It's a monotonic divergence.

Logical integration. if prod P(x)dx = e^{int ln(P(x))dx} = 1, 

The issue with real induction where you can push the ball out a decreasing amoutn is similar to the concerns that led cantor to invent the orindals in the first place.

It's also like finite escape vs asymptitotic escape/ lyapunov coefficients.

Graham says Freyd reals. A double pointed set with gluing? Kind of like an interval definition

Hamkins - The problem is the prefixes aren't determined by prefixes


In [ ]:
I = Datatype("Interval")
I.

In [ ]:
RFun = ArraySort(R,R)
cont = Function(RFun, B)

cont_def = ForAll([f], cont(f) == ForAll([eps], Exists([N], ForAll([n], Implies(n >= N, abs(f(n) - f(0)) < eps)))))


In [105]:
# RecFunction allows us to use z3 as a simplifier.

#sum__ = RecFunction("sum", Seq, Nat, R)
#RecAddDefinition(sum__, [s, n], If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum__(s, Nat.pred(n)))))
#simplify(sum__(Lambda([n], If(Nat.is_zero(n), RealVal(0), RealVal(1))), Nat.succ(Nat.zero)))

#sum_id = RecFunction("sum_id", Seq, Nat, R)
#RecAddDefinition(sum_id, [n], If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum__(s, Nat.pred(n)))))
#simplify(sum__(Lambda([n], If(Nat.is_zero(n), RealVal(0), RealVal(1))), Nat.succ(Nat.zero)))

real_ = RecFunction("real", Nat, R)
RecAddDefinition(real_, [n], If(Nat.is_zero(n), 0, 1 + real_(Nat.pred(n))))
print(simplify(real_(Nat.succ(Nat.succ(Nat.zero)))))
simplify(real_(Nat.succ(Nat.succ(n))))


2


real(succ(succ(n)))

Nets. Sequences are perhaps arbitrary,


Filters. Havibng a domain at all is perhaps arbitrary


Math comp analyses and zach was all about filters.



In [71]:
%%file /tmp/induct.smt2
(declare-sort Nat 0)
(declare-fun sum ((Array Nat Real) Nat) Real)
(declare-fun real (Nat) Real)
(declare-fun plus (Real Real) Real)
(declare-fun mul (Real Real) Real)
(assert (let ((a!1 (forall ((x Real))
             (= (sum ((as const (Array Nat Real)) x) zero) 0.0)))
      (a!2 (forall ((n Nat))
             (let ((a!1 (forall ((x Real))
                          (= (sum ((as const (Array Nat Real)) x) n)
                             (mul (real n) x))))
                   (a!2 (forall ((x Real))
                          (= (sum ((as const (Array Nat Real)) x) (succ n))
                             (mul (real (succ n)) x)))))
               (=> a!1 a!2))))
      (a!3 (forall ((x Real))
             (= (sum ((as const (Array Nat Real)) x) zero) (mul (real zero) x))))
      (a!4 (forall ((n Nat))
             (forall ((x Real))
               (= (sum ((as const (Array Nat Real)) x) n) (mul (real n) x)))))
      (a!5 (forall ((x Real) (n Nat))
             (= (sum ((as const (Array Nat Real)) x) n) (mul (real n) x)))))
  (=> (and a!1 a!2 (=> (and a!3 a!2) a!4)) a!5)))

(check-sat)

Overwriting /tmp/induct.smt2


In [70]:
! vampire /tmp/induct.smt2

% Running in auto input_syntax mode. Trying SMTLIB2
% Failed with
% User error: Compound functor expected to be a rankend function (starting with '_'). Instead read: (as const (Array Nat Real))
% Trying TPTP
Parsing Error on line 1: cnf(), fof(), vampire() or include() expected at position 0 (text: ()
